In [2]:
import numpy as np
import pandas as pd
import pickle

In [3]:
laminate = pd.read_csv('laminates.csv')

In [4]:
laminate.head()

,design_no,laminate_code,page_no,assigned_value,finish_name,laminate
0,1322,AB,54.0,1322,NaN,verticalwood
1,1466,AL,45.0,1466,NaN,verticalwood
2,1161 (3),AR,28.0,11612,NaN,stone solidcolor
3,1162 (2),AR,28.0,11622,glossy,stone solidcolor
4,1620,AR,28.0,1620,NaN,stone


In [5]:
laminates = laminate[['design_no','laminate_code','assigned_value','laminate']]

In [6]:
laminates.head()

,design_no,laminate_code,assigned_value,laminate
0,1322,AB,1322,verticalwood
1,1466,AL,1466,verticalwood
2,1161 (3),AR,11612,stone solidcolor
3,1162 (2),AR,11622,stone solidcolor
4,1620,AR,1620,stone


In [7]:
laminates.dropna

<bound method DataFrame.dropna of     design_no laminate_code  assigned_value                 laminate
0        1322            AB            1322             verticalwood
1        1466            AL            1466             verticalwood
2    1161 (3)            AR           11612         stone solidcolor
3    1162 (2)            AR           11622         stone solidcolor
4        1620            AR            1620                    stone
..        ...           ...             ...                      ...
886      1593            WB            1593  verticalwood exoticwood
887      1600            WB            1600  verticalwood exoticwood
888   132 (9)            WD          132100  verticalwood solidcolor
889   583 (2)            WD            5831             verticalwood
890   992 (2)            WD            9921             verticalwood

[891 rows x 4 columns]>

In [8]:
import sklearn

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
tfid = TfidfVectorizer(max_features=100)

In [11]:
laminates['laminate']=laminates['laminate'].fillna(' ')

C:\Users\Dell\AppData\Local\Temp\ipykernel_1064\265461977.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  laminates['laminate']=laminates['laminate'].fillna(' ')


In [12]:
tfid_matrix=tfid.fit_transform(laminates['laminate'])

In [13]:
tfid_matrix.shape

(891, 9)

In [14]:
tfid_matrix

<891x9 sparse matrix of type '<class 'numpy.float64'>'
	with 937 stored elements in Compressed Sparse Row format>

In [15]:
from sklearn.metrics.pairwise import sigmoid_kernel

In [16]:
sig = sigmoid_kernel(tfid_matrix, tfid_matrix)

In [17]:
indices = pd.Series(laminates.index, index=laminates['assigned_value']).drop_duplicates()

In [18]:
def recommend(title, sig=sig):
    idx = indices[title]
    sig_scores = list(enumerate(sig[idx]))
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)
    sig_scores = sig_scores[1:11]
    laminate_indices = [i[0] for i in sig_scores]
    return laminates.iloc[laminate_indices]


In [19]:
recommend(7005)

,design_no,laminate_code,assigned_value,laminate
18,1642,BG,1642,stone vibrant
182,1201,CRYSTAL,1201,stone vibrant
183,1202,CRYSTAL,1202,stone vibrant
194,1213 (2),CRYSTAL,12131,stone vibrant
196,1215 (2),CRYSTAL,12151,stone vibrant
202,1227 (2),CRYSTAL,12271,stone vibrant
228,1264 (2),CRYSTAL,12641,stone vibrant
277,1215 (2),CRYSTAL Z+BG,12152,stone vibrant
278,1240 (2),CRYSTAL Z+BG,12402,stone vibrant
319,1213 (2),CRYSTAL Z+SV,12132,stone vibrant


In [22]:
filename="royaletouche_pkl.pkl"
pickle.dump(laminates,open(filename,'wb'))

In [20]:
pickle.dump(indices,open('indicesx','wb'))

In [21]:
pickle.dump(sig,open('sigx','wb'))